In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy import io
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm
import time
from datetime import datetime
import os
import random
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from dolfin import *
from mshr import *
import math

/home/haltmayermarc/anaconda3/envs/fenics/lib/python3.11/site-packages/dolfin/jit/jit.py:121: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
  def compile_class(cpp_data, mpi_comm=MPI.comm_world):


In [5]:
class RandomNoSlipIC(UserExpression):
    def __init__(self, ks, ls, amps, degree=4, **kwargs):
        super().__init__(degree=degree, **kwargs)
        self.ks = np.array(ks, dtype=int)
        self.ls = np.array(ls, dtype=int)
        self.amps = np.array(amps, dtype=float)
        self.pi = math.pi

    def eval(self, values, x):
        X, Y = float(x[0]), float(x[1])
        ux, uy = 0.0, 0.0
        for k, l, a in zip(self.ks, self.ls, self.amps):
            sx = math.sin(k * self.pi * X)
            sy = math.sin(l * self.pi * Y)
            cx = math.cos(k * self.pi * X)
            cy = math.cos(l * self.pi * Y)
            # u = (∂ψ/∂y, -∂ψ/∂x), ψ = sin²(kπx) sin²(lπy)
            ux += a * (2.0 * (sx * sx) * sy * cy * l * self.pi)
            uy += a * (-2.0 * sx * cx * (sy * sy) * k * self.pi)
        values[0] = ux
        values[1] = uy

    def value_shape(self):
        return (2,)

def random_ic_expression(n_modes=3, kmax=4, lmax=4, amp=1.0, seed=None, degree=4):
    rng = np.random.default_rng(seed)
    ks = rng.integers(1, kmax+1, size=n_modes)
    ls = rng.integers(1, lmax+1, size=n_modes)
    scales = (ks**2 + ls**2)**(-0.5)  # decay to keep smooth
    amps = amp * rng.normal(size=n_modes) * scales
    return RandomNoSlipIC(ks=ks, ls=ls, amps=amps, degree=degree)

def u_ini_0(m0, n0, grid):
  grid=torch.tensor(grid)
  return -5.0+m0*torch.sin(n0*grid[:,0])*torch.sin(grid[:,1])

def u_ini_1(m1, n1, grid):
  grid=torch.tensor(grid)
  return 0.0+m1*torch.cos(n1*grid[:,0])*torch.sin(grid[:,1])

In [2]:
dt = 0.1
num_xy = 10
T=[0,1]
grid_t=np.arange(T[0],T[1]+dt,dt)

mesh = RectangleMesh(Point(0, 0), Point(1,1), num_xy, num_xy)

# Define the function spaces:
V = VectorElement('CG', triangle, 2)
Q = FiniteElement('CG', triangle, 1)
TH = V * Q
W = FunctionSpace(mesh, TH)

# Define the Dirichlet BC
SlipRate = Expression ( ( "-5.0", "0.0" ), degree=3)
def LowerBoundary ( x, on_boundary ):
    return x[1] < DOLFIN_EPS and on_boundary
bc = DirichletBC(W.sub ( 0 ), SlipRate, LowerBoundary)
#zero = Constant((0.0, 0.0))   # if W.sub(0) is vector-valued
#bc = DirichletBC(W.sub(0), zero, 'on_boundary')

#  Define the variational problem
( u, p ) = TrialFunctions ( W )
( v, q ) = TestFunctions ( W )
#  Material viscosity, Pa-sec.
mu=1
a = ( mu * inner(grad(v), grad(u)) \
    - div ( v )* p + q * div ( u ) ) * dx
f = Constant ( ( 5.0, -5.0) )
l = inner ( v, f ) * dx

w = Function(W)

#  Matrix assembly.
## We will do (S/dt+A) @ newcoeff = (S/dt) @ old_coeff + load_vector
## Therefore, newcoeff = (S/dt+A)^-1 @ [(S/dt) @ old_coeff + load_vector]
s = inner(v,u) * dx
S = assemble(s)
bc.apply(S)
S = S.array()

A = assemble(a)
bc.apply(A)
A = A.array()

L = assemble(l)
bc.apply(L)
load_vector = L.get_local()

ne=mesh.cells().shape[0]

# FENiCS ordering
pos_u1=W.sub(0).sub(0).collapse().tabulate_dof_coordinates()
pos_u2=W.sub(0).sub(1).collapse().tabulate_dof_coordinates()
pos_p=W.sub(1).collapse().tabulate_dof_coordinates()

# numerical ordering
pos_all=W.tabulate_dof_coordinates()
idx_u1=W.sub(0).sub(0).dofmap().dofs()
idx_u2=W.sub(0).sub(1).dofmap().dofs()
idx_p=W.sub(1).dofmap().dofs()

# Construct the permutations for re-indexing
row_to_index_u1 = {tuple(row): i for i, row in enumerate(pos_u1)}
row_to_index_u2 = {tuple(row): i for i, row in enumerate(pos_u2)}
row_to_index_p = {tuple(row): i for i, row in enumerate(pos_p)}
perm_u1 = np.array([row_to_index_u1[tuple(row)] for row in pos_all[idx_u1]])
perm_u2 = np.array([row_to_index_u2[tuple(row)] for row in pos_all[idx_u2]])
perm_p = np.array([row_to_index_p[tuple(row)] for row in pos_all[idx_p]])
  
ng=pos_all.shape[0]

idx_bdry0_pts=list(bc.get_boundary_values().keys())

gfl = np.zeros((ng,1))
gfl[idx_bdry0_pts]=1
  
idx_sol=[idx_u1,idx_u2,idx_p]
idx_sol=np.array(idx_sol, dtype=object)

/home/haltmayermarc/anaconda3/envs/fenics/lib/python3.11/site-packages/dolfin/jit/jit.py:46: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
  if MPI.size(mpi_comm) == 1:


In [3]:
#u_init = random_ic_expression(n_modes=5, kmax=4, lmax=4, amp=1.0, seed=5, degree=4)
m0, m1 = 2+np.random.rand(2)
n0, n1= 2*np.pi*(np.random.rand(2))
u_init = Expression ( ( "-5.0+m0*sin(n0*x[0])*sin(x[1])", "0.0+m1*cos(n1*x[0])*sin(x[1])" ), degree=5, m0=m0, m1=m1, n0=n0, n1=n1 )
u_b = project(u_init, W.sub ( 0 ).collapse())
u_b_vector = u_b.vector()[:]
u_b_vector_x = u_b.sub(0, deepcopy=True).vector()[:]
u_b_vector_y = u_b.sub(1, deepcopy=True).vector()[:]

DT = Constant(dt)
u_t = (1/DT) * inner ( v, (u-u_b) ) * dx

F = u_t + a - l

LHS = lhs(F)
RHS = rhs(F)

solve(LHS==RHS, w, bc)
(u, p) = w.split(deepcopy=True)
sol_u1=u.sub(0, deepcopy=True).vector()[:]
sol_u2=u.sub(1, deepcopy=True).vector()[:]
sol_p=p.vector()[:]

Solving linear variational problem.


In [6]:
u_test_pos_u2 = np.array(u_ini_1(m1,n1,pos_u2))
u_test_pos_all = np.array(u_ini_1(m1,n1,pos_all[idx_sol[1]]))
u_test_pos_all_ordered = np.array(u_ini_1(m1,n1,pos_all[idx_u2]))[perm_u2]
u_b_vector_y_ordered = u_b_vector_y[perm_u2]

print(np.linalg.norm(u_b_vector_y - u_test_pos_u2))
print(np.linalg.norm(u_b_vector_y - u_test_pos_all))
print(np.linalg.norm(u_b_vector_y - u_test_pos_all_ordered))
print("#########")
print(np.linalg.norm(u_b_vector_y_ordered - u_test_pos_u2))
print(np.linalg.norm(u_b_vector_y_ordered - u_test_pos_all))
print(np.linalg.norm(u_b_vector_y_ordered - u_test_pos_all_ordered))

0.0002775316833592231
0.1905345399666986
0.08948881492754213
#########
0.19054129939131476
0.0002775316833592231
0.1905345399666986


/tmp/ipykernel_683923/2566274105.py:1: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  u_test_pos_u2 = np.array(u_ini_1(m1,n1,pos_u2))
/tmp/ipykernel_683923/2566274105.py:2: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  u_test_pos_all = np.array(u_ini_1(m1,n1,pos_all[idx_sol[1]]))
/tmp/ipykernel_683923/2566274105.py:3: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword a

In [8]:
u_test_pos_u2.shape

(441,)

In [9]:
pos_u2.shape

(441, 2)

## Linalg

In [18]:
u_solve_linalg=np.zeros(len(idx_sol[0])+len(idx_sol[1])+len(idx_sol[2]))
u_solve_linalg[idx_sol[0]]=u_b_vector_x
u_solve_linalg[idx_sol[1]]=u_b_vector_y
u_solve_linalg = torch.tensor(u_solve_linalg)
S = torch.tensor(S)
A = torch.tensor(A)
load_vector = torch.tensor(load_vector)
predict=torch.linalg.solve(S+A*dt,torch.matmul(S,u_solve_linalg.reshape(-1,1))+dt*load_vector.reshape(-1,1))
predict = np.array(predict)

/tmp/ipykernel_2430471/3609901792.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  S = torch.tensor(S)
/tmp/ipykernel_2430471/3609901792.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  A = torch.tensor(A)
/tmp/ipykernel_2430471/3609901792.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  load_vector = torch.tensor(load_vector)
/tmp/ipykernel_2430471/3609901792.py:9: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword argument

In [21]:
predict[idx_sol[0]] - sol_u1

array([[ 0.01290498,  0.01290498,  0.01290498, ...,  0.01290498,
         0.01290498,  0.01290498],
       [ 0.10560651,  0.10560651,  0.10560651, ...,  0.10560651,
         0.10560651,  0.10560651],
       [-0.09195643, -0.09195643, -0.09195643, ..., -0.09195643,
        -0.09195643, -0.09195643],
       ...,
       [-0.01290498, -0.01290498, -0.01290498, ..., -0.01290498,
        -0.01290498, -0.01290498],
       [-0.05544543, -0.05544543, -0.05544543, ..., -0.05544543,
        -0.05544543, -0.05544543],
       [ 0.03111267,  0.03111267,  0.03111267, ...,  0.03111267,
         0.03111267,  0.03111267]], shape=(441, 441))

In [22]:
sol_u1_ordered = sol_u1[perm_u1]

In [23]:
predict[idx_sol[0]] - sol_u1_ordered

array([[ 0.01290498,  0.01290498,  0.01290498, ...,  0.01290498,
         0.01290498,  0.01290498],
       [ 0.10560651,  0.10560651,  0.10560651, ...,  0.10560651,
         0.10560651,  0.10560651],
       [-0.09195643, -0.09195643, -0.09195643, ..., -0.09195643,
        -0.09195643, -0.09195643],
       ...,
       [-0.01290498, -0.01290498, -0.01290498, ..., -0.01290498,
        -0.01290498, -0.01290498],
       [-0.05544543, -0.05544543, -0.05544543, ..., -0.05544543,
        -0.05544543, -0.05544543],
       [ 0.03111267,  0.03111267,  0.03111267, ...,  0.03111267,
         0.03111267,  0.03111267]], shape=(441, 441))